In [1]:
import awswrangler as wr
from datetime import datetime, timedelta
import sagemaker
import boto3
import json
import subprocess

now = datetime.now() - timedelta(hours = 4)

# episodes = ["1716761117540","test"] # pageids of episodes to take
# programs = ["1460018486818"] # program pageids of programs to take

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/willem/.config/sagemaker/config.yaml


In [2]:
# Login AWS
aws_profile = "vrt-analytics-engineer-nonsensitive"
envvars = subprocess.check_output(['aws-vault', 'exec', aws_profile, '--', 'env'])
for envline in envvars.split(b'\n'):
    line = envline.decode('utf8')
    eqpos = line.find('=')
    if eqpos < 4:
        continue
    k = line[0:eqpos]
    v = line[eqpos+1:]
    if k == 'AWS_ACCESS_KEY_ID':
        aws_access_key_id = v
    if k == 'AWS_SECRET_ACCESS_KEY':
        aws_secret_access_key = v
    if k == 'AWS_SESSION_TOKEN':
        aws_session_token = v

session = boto3.Session(
aws_access_key_id, aws_secret_access_key, aws_session_token, region_name = "eu-west-1"
)
credentials = session.get_credentials()
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

credentials =  {
    "boto_session": session,
    "sagemaker_session": sagemaker_session,
    "role": role
}

In [3]:
query = f"""
SELECT
* 
FROM (
SELECT
*
FROM (
SELECT 
*
FROM 
derived_prod.vrtmax_catalog_mediaid_history WHERE year = {now.year} and month = {now.month} and day = {now.day} and hour = {now.hour} 
AND offering_publication_planneduntil > CURRENT_TIMESTAMP
) vrtmax_catalog
LEFT JOIN (
SELECT 
mediacontent_media_id as mediacontent_media_id_duplicate,
array_join(array_agg(subtitle ORDER BY starttc ASC), ' ') as subtitle
FROM (
SELECT 
MIN(subtitle) as subtitle,
mediacontent_media_id,
starttc
FROM (
SELECT 
subtitle,
mediacontent_media_id,
starttc
FROM
(
SELECT 
objid, subtitle
FROM (
SELECT objid, subtitle, 
row_number() OVER (partition by subtitle ORDER BY objid DESC) as rn 
FROM (
SELECT objid, a_vrt_subti2  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti3  as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subtit as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s3"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid, a_vrt_subti4 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_s4"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_speech as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_sp"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION 
SELECT objid, a_vrt_subti1 as subtitle FROM "operationaldata_nonsensitive_prod"."depot_dmc_s_vrt_su"   -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
)
WHERE rn = 1
) a
LEFT JOIN (
SELECT objid, segmentcontentid, starttc
FROM (
SELECT objid, segmentcontentid, starttc,
row_number() OVER (partition by segmentcontentid ORDER BY objid DESC) as rn 
FROM (
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub1"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub2"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub3"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_sub4"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
UNION
SELECT objid,  segmentcontentid ,starttc FROM "operationaldata_nonsensitive_prod"."depot_dms_vrt_subt"  -- WHERE import_year = 2024 and import_month = 6 and import_day = 9
)
GROUP BY objid, segmentcontentid, starttc
) 
WHERE rn = 1
) b
ON a.objid = b.segmentcontentid
LEFT JOIN (
SELECT objid,  a_vrt_progra ,a_vrt_media_
FROM (
SELECT objid,  a_vrt_progra ,a_vrt_media_,
row_number() OVER (partition by objid ORDER BY a_vrt_media_ DESC) as rn 
FROM "operationaldata_nonsensitive_prod"."depot_dmo_program"  WHERE import_year = 2024 and import_month = 6 and import_day = 9
) WHERE rn = 1
) c
ON b.objid = c.objid
INNER JOIN (
SELECT mediacontent_media_id, 	mediacontent_media_origin_id  
FROM (
SELECT mediacontent_media_id, mediacontent_media_origin_id, row_number() OVER (PARTITION BY mediacontent_media_origin_id   ORDER BY 	contactmoment_ingest_timestamp DESC)  as rn FROM "marketing_prod"."dtr_vd_dms_audio_video_metadata_datariver"
)
WHERE rn = 1
) d
ON c.a_vrt_media_ = d.mediacontent_media_origin_id
)
GROUP BY mediacontent_media_id, starttc
)
GROUP BY mediacontent_media_id
) subtitles  
ON vrtmax_catalog.mediacontent_media_id = subtitles.mediacontent_media_id_duplicate
) intermediary
LEFT JOIN (
SELECT
mediacontent_pageid_duplicate,mediacontent_episode_castlist
FROM (
SELECT 
mediacontent_pageid as mediacontent_pageid_duplicate,mediacontent_episode_castlist,
row_number() over (partition by mediacontent_pageid ORDER BY mediacontent_page_modificationdate desc) as rn
FROM marketing_prod.aem_vrtvideo_datariver
WHERE year = {now.year}  and mediacontent_episode_castlist is not null )
WHERE rn = 1
) castinfo ON intermediary.mediacontent_pageid = castinfo.mediacontent_pageid_duplicate
"""

df = wr.athena.read_sql_query(sql=query, database="derived_prod",boto3_session=credentials["boto_session"])

In [4]:
# drop columns
df = df.drop(['mediacontent_media_id_duplicate','mediacontent_pageid_duplicate'], axis=1)

In [5]:
df

,mediacontent_media_id,mediacontent_pageid,mediacontent_season_pageid,mediacontent_program_pageid,offering_publication_plannedfrom,offering_publication_planneduntil,brand_contentbrand,mediacontent_media_origin_id,mediacontent_episode_age,mediacontent_episode_number,...,mediacontent_media_type,mediacontent_media_content_type,mediacontent_recommendable,mediacontent_page_editorialtitle_program,year,month,day,hour,subtitle,mediacontent_episode_castlist
0,vid-8deb6967-9d94-4acc-8839-2cfbe60a5047,1529657106876,1529657860887,1529657844167,2018-06-27 21:59:00,2025-12-31 22:55:00,klara,WP00135208,<NA>,25,...,full,video,True,Pat Donnez legt psychiater Edel Maex zesentwin...,2024,9,19,14,<NA>,None
1,vid-596d4eba-e7ce-4ee9-879f-9613f80bdf11,1709820005710,<NA>,1709819792176,2024-03-29 05:00:00,2025-03-21 22:55:00,canvas,WP00257916,<NA>,3,...,<NA>,video,True,Tim Verheyden onderzoekt de impact van technol...,2024,9,19,14,"Ik ben in Boston, onderweg naar het MIT, het ...",None
2,vid-8444e780-d430-40ae-b512-6a0caed7e647,1725631796302,1724058789849,1724057969608,2024-09-06 14:08:00,2034-09-06 14:08:00,ketnet,0fcaa9f1-b5f6-4f45-942b-734f21792d20,<NA>,<NA>,...,full,video,True,<NA>,2024,9,19,14,<NA>,None
3,vid-6447862c-aed8-4761-86bf-da73f2e4660a,1508942316557,1508851428140,1508839995749,2018-06-18 06:00:00,2025-12-31 22:55:00,klara,WP00121814,<NA>,16,...,full,video,True,Ruth Joos legt romanschrijver Stefan Hertmans ...,2024,9,19,14,<NA>,None
4,vid-654e61f5-61e3-42f1-aa40-f49c7bf699ec,1597665932654,1597237912628,1459955889901,2020-11-29 05:00:00,2025-12-31 22:59:00,een,E03009847,<NA>,133,...,full,video,True,Vlaanderens populairste dagelijkse fictiereeks,2024,9,19,14,<NA>,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22455,vid-919f8162-c307-44ee-b4dd-deb4a94519c6,1581352206174,1580986458340,1488793670409,2020-02-11 05:00:00,2025-12-31 22:59:00,radio1,WP00173182,<NA>,2,...,full,video,True,Volledige live muzieksessies van Radio 1,2024,9,19,14,<NA>,None
22456,vid-3b18d8bd-3b5e-4244-85e0-66de1fe2d367,1465304350426,1460120597239,1460120593552,2023-06-06 04:00:00,2030-06-01 21:59:00,ketnet,WP00012861,<NA>,12,...,full,video,True,Deze jonge vrienden zoeken hun weg in het leven,2024,9,19,14,Je kan zoveel bijstuderen. Voor mij zit er nie...,None
22457,vid-bd02acc8-06c7-4dc9-93c4-fa0475baad29,1500388639966,1487952605630,1459955889901,2020-12-17 05:00:00,2025-12-31 22:59:00,een,WP00049880,<NA>,3659,...,full,video,True,Vlaanderens populairste dagelijkse fictiereeks,2024,9,19,14,"Je bent niet goed bezig, Peggy. Wat jij allema...",None
22458,vid-c5070b45-3766-48ae-9e67-a94745385428,1693398650570,1693398650423,1693398425377,2023-09-06 04:00:00,2024-09-26 21:59:00,vrt-nu,WP00246568,<NA>,3,...,full,video,True,Koppels praten over hun IVF-traject,2024,9,19,14,Jullie voelden zich vaak eerder een medisch d...,None


In [6]:
df[df.mediacontent_pagetitle_program.str.contains("fspraak")].iloc[2]

mediacontent_media_id                                  vid-b74a5949-d96d-4c6f-870b-1ff33696bb84
mediacontent_pageid                                                               1724105108987
mediacontent_season_pageid                                                        1703459125991
mediacontent_program_pageid                                                       1460018486818
offering_publication_plannedfrom                                     2024-09-03 18:33:07.120000
offering_publication_planneduntil                                           2024-10-03 21:55:00
brand_contentbrand                                                                       vrtnws
mediacontent_media_origin_id                                                         WP00259248
mediacontent_episode_age                                                                   <NA>
mediacontent_episode_number                                                                  69
mediacontent_episode_region             

In [10]:
# cleaning
df["subtitle"] = df["subtitle"].str.replace('\r', '')
df["subtitle"] = df["subtitle"].str.replace('\n', '')
df["subtitle"] = df["subtitle"].str.replace("-","")

# Fields to use
fields_to_embed = [
    "subtitle","mediacontent_page_description_program","mediacontent_page_description","mediacontent_page_editorialtitle_program"
]

metadata_fields = [
    "mediacontent_pagetitle_program","mediacontent_pagetitle_season","mediacontent_pagetitle"
    ,"offering_publication_planneduntil","brand_contentbrand","mediacontent_pageurl","mediacontent_imageurl","mediacontent_programimageurl",
    "mediacontent_episode_castlist","mediacontent_media_content_type","mediacontent_page_editorialtags"
]

# convert everythin to string 
df = df[fields_to_embed+metadata_fields].astype(str)
df.fillna('', inplace=True)
df = df.replace('<NA>', "")
df = df.replace('"', "")

In [2]:
""" # take only  rows where at least one of the fields to be embedded is not null
query_condition = []

for field in fields_to_embed:
    query_condition.append("(" + field + "!= ''")

query_condition = " | ".join(query_condition)
df = df.query(query_condition) """

' # take only  rows where at least one of the fields to be embedded is not null\nquery_condition = []\n\nfor field in fields_to_embed:\n    query_condition.append("(" + field + "!= \'\'")\n\nquery_condition = " | ".join(query_condition)\ndf = df.query(query_condition) '

In [11]:
# loading data into vector db requires one field with all text data to embed
df["info_to_embed"] = df[fields_to_embed].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# rest of columns become metadata automatically, so remove the ones you don't want
df = df.drop(['subtitle'], axis=1)

In [18]:
import csv
df.to_csv("vrtmax_catalog.csv",index=False)

In [40]:
# create csv + metadata combinations (for every row)
for index, row in df.iterrows():
    df[df.index == index][fields_to_embed].to_csv("opensearch/" + row["mediacontent_media_id"] + ".csv",index=False)
    metadata = dict()
    metadata["metadataAttributes"] = df[df.index == index][metadata_fields].to_dict("records")[0]

    # Convert and write JSON object to file
    with open("opensearch/" + row["mediacontent_media_id"] + ".metadata.json", "w") as outfile: 
        json.dump(metadata, outfile)